# Final Project for DTSA 5304 - Intro to Data Visualizations at the University of Colorado Masters of Data Science Degree

## This workbook will serve as an analysis of all Pokemon stats through Generation 6, which was released in 2013. 

## This analysis will be done in Python3, using the Altair library for the Visualizations

In [1]:
import pandas as pd
import altair as alt
import numpy as np
import requests
from io import StringIO


url = 'https://raw.githubusercontent.com/rdadmun/Data_Science_Degree/main/DTSA_5304/Pokemon_Stats.csv'
response = requests.get(url)
pkmn_data = pd.read_csv(StringIO(response.text))
pkmn_data.head()

/var/folders/d8/j2s681kd64q1hp65blg721040000gn/T/ipykernel_12116/369857577.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


ModuleNotFoundError: No module named 'requests'

Next, let us clean the variables a little bit, by following standard variable naming conventions and replacing the ". " with  an underscore in our variables. 

In [ ]:
#Edit Variable Names
pkmn_data = pkmn_data.rename(columns={'Sp. Atk': 'Sp_Atk', 'Sp. Def': 'Sp_Def'})

#Remove Mega Pokemon
pkmn_data = pkmn_data[~pkmn_data['Name'].str.contains('Mega', case=False)]
print(pkmn_data)

       #                 Name   Type 1  Type 2  Total   HP  Attack  Defense  \
0      1            Bulbasaur    Grass  Poison    318   45      49       49   
1      2              Ivysaur    Grass  Poison    405   60      62       63   
2      3             Venusaur    Grass  Poison    525   80      82       83   
4      4           Charmander     Fire     NaN    309   39      52       43   
5      5           Charmeleon     Fire     NaN    405   58      64       58   
..   ...                  ...      ...     ...    ...  ...     ...      ...   
794  718     Zygarde50% Forme   Dragon  Ground    600  108     100      121   
795  719              Diancie     Rock   Fairy    600   50     100      150   
797  720  HoopaHoopa Confined  Psychic   Ghost    600   80     110       60   
798  720   HoopaHoopa Unbound  Psychic    Dark    680   80     160       60   
799  721            Volcanion     Fire   Water    600   80     110      120   

     Sp_Atk  Sp_Def  Speed  Generation  Legendary  

Next we will define colors for each of our various pokemon types throughout our graphs

In [ ]:
#Defining Colors
colors = {
    "Bug": "#ABC206",
    "Dark": "#4A3A2F",
    "Dragon": "#5F21F6",
    "Electric": "#E7D711",
    "Fairy": "#EC83B7",
    "Fighting": "#C6231C",
    "Fire": "#F57C22",
    "Flying": "#A58CEB",
    "Ghost": "#684E8A",
    "Grass": "#76C945",
    "Ground": "#D6B55E",
    "Ice": "#9CE0DD",
    "Normal": "#B3B288",
    "Poison": "#B23BAF",
    "Psychic": "#FF467E",
    "Rock": "#B6A136",
    "Steel": "#C5C5D3",
    "Water": "#4C7CE2",
}

First, lets see how many of each type of pokemon there are, based solely on their primary typing

In [ ]:
#Primary Type Pie Chart
selection = alt.selection(type="multi", fields=["Pokemon Primary Type"])
base =  alt.Chart(pkmn_data).properties(width=500, height=350)

base = alt.Chart(pkmn_data).encode(
    alt.Theta("count(Type 1)").stack(True),
    color=alt.Color("Type 1", legend = None, scale=alt.Scale(range=list(colors.values())))
)

pie = base.mark_arc(innerRadius=0)
text = base.mark_text(radius = 190, size = 12).encode(text="Type 1:N")
text2 = base.mark_text(radius = 162, size = 12).encode(text="count(Type 1)")

pie + text + text2

/Users/ryandadmun/venv/lib/python3.12/site-packages/altair/utils/deprecation.py:65: AltairDeprecationWarning: 'selection' is deprecated.
   Use 'selection_point()' or 'selection_interval()' instead; these functions also include more helpful docstrings.
  warnings.warn(message, AltairDeprecationWarning, stacklevel=1)
/Users/ryandadmun/venv/lib/python3.12/site-packages/altair/vegalite/v5/api.py:469: AltairDeprecationWarning: The types 'single' and 'multi' are now
        combined and should be specified using "selection_point()".
  warnings.warn(


alt.LayerChart(...)

This is a pretty good distribution and visualization for primary typings! Let's create a second one for secondary typings as well. As not every pokemon has a secondary type, we will create a new dataset where the null secondary typings have been filtered out.

In [ ]:
#Secondary Pokemon Type Pie Chart
pkmn_data_filtered = pkmn_data.dropna(subset=['Type 2'])

selection = alt.selection(type="multi", fields=["Pokemon Primary Type"])
base =  alt.Chart(pkmn_data_filtered).properties(width=500, height=350)

base = alt.Chart(pkmn_data_filtered).encode(
    alt.Theta("count(Type 2)").stack(True),
    color=alt.Color("Type 2", legend = None, scale=alt.Scale(range=list(colors.values())))
)

pie = base.mark_arc(innerRadius=0)
text = base.mark_text(radius = 190, size = 12).encode(text="Type 2:N")
text2 = base.mark_text(radius = 162, size = 12).encode(text="count(Type 2)")

pie + text + text2

/Users/ryandadmun/venv/lib/python3.12/site-packages/altair/utils/deprecation.py:65: AltairDeprecationWarning: 'selection' is deprecated.
   Use 'selection_point()' or 'selection_interval()' instead; these functions also include more helpful docstrings.
  warnings.warn(message, AltairDeprecationWarning, stacklevel=1)
/Users/ryandadmun/venv/lib/python3.12/site-packages/altair/vegalite/v5/api.py:469: AltairDeprecationWarning: The types 'single' and 'multi' are now
        combined and should be specified using "selection_point()".
  warnings.warn(


alt.LayerChart(...)

This is a decent visualization for how the types of Pokemon are distributed across the whole series, but I would like to create a different visualization for the types of Pokemon across Generations of the series. 

In [ ]:
# Group by "Type 1" and calculate counts for each generation
type_counts = pkmn_data.groupby(['Generation', 'Type 1']).size().reset_index(name='Count')

# Create the line chart
Gen_line_chart = alt.Chart(type_counts).mark_line(point=True, strokeWidth=2).encode(
    x='Generation:N',
    y='Count:Q',
    color=alt.Color("Type 1", scale=alt.Scale(range=list(colors.values()))),
    tooltip=['Generation:N', 'Type 1:N', 'Count:Q']
).properties(
    title='Counts per Pokémon across Generations per type',
    width=600, 
    height=400 
).configure_point(
    size=80
)

# Show the line chart
Gen_line_chart

alt.Chart(...)

Now that we have some visualizations for the distribution of pokemon types, lets explore the Stats a bit!

Pokemon have a total of 6 stats: HP, Attack, Defense, Special Attack, Special Defense and Speed.
- HP: Hit Points, or Pokemon Health
- Attack: Used in calculation for physical moves damage
- Defense: Used in the calculation for physical moves damage.
- Special Attack: Used in the calculation for non-physical moves damage.
- Special Defense: Used in the calculation for non-physical moves damage.
- Speed: Used to determine the turn order in a Pokemon Battle.

First, lets create a Bar Chart of Pokemon types and the means of their respective stats to see how they compare against each other.

In [ ]:
selection = alt.selection(type="multi", fields=["Pokemon Primary Type"])
base =  alt.Chart(pkmn_data).properties(width=500, height=250)

#HP Chart
HP_Bar = alt.Chart(pkmn_data).mark_bar().encode(
    y = "mean(HP)",
    x = alt.X("Type 1", title ="Primary Type"),
    color=alt.Color("Type 1", legend=None, scale=alt.Scale(range=list(colors.values()))),
    tooltip=["Type 1", "mean(HP)"],
    opacity=alt.condition(selection,alt.value(1),alt.value(.2))
).add_selection(selection).properties(height=250, width=250)

#Attack Chart
Atk_Bar = alt.Chart(pkmn_data).mark_bar().encode(
    y = "mean(Attack)",
    x = alt.X("Type 1", title ="Primary Type"),
    color=alt.Color("Type 1", legend=None, scale=alt.Scale(range=list(colors.values()))),
    tooltip=["Type 1", "mean(Attack)"],
).add_selection(selection).properties(height=250, width=250)

#Defense Chart
Def_Bar = alt.Chart(pkmn_data).mark_bar().encode(
    y = "mean(HP)",
    x = alt.X("Type 1", title ="Primary Type"),
    color=alt.Color("Type 1", legend=None, scale=alt.Scale(range=list(colors.values()))),
    tooltip=["Type 1", "mean(Defense)"],
).add_selection(selection).properties(height=250, width=250)

#Sp. Atk Chart
SpAtk_Bar = alt.Chart(pkmn_data).mark_bar().encode(
    y = "mean(Sp_Atk)",
    x = alt.X("Type 1", title ="Primary Type"),
    color=alt.Color("Type 1", legend=None, scale=alt.Scale(range=list(colors.values()))),
    tooltip=["Type 1", "mean(Sp_Atk)"],
).add_selection(selection).properties(height=250, width=250)

#Sp. Def Chart
SpDef_Bar = alt.Chart(pkmn_data).mark_bar().encode(
    y = "mean(Sp_Def)",
    x = alt.X("Type 1", title ="Primary Type"),
    color=alt.Color("Type 1", legend=None, scale=alt.Scale(range=list(colors.values()))),
    tooltip=["Type 1", "mean(Sp_Def)"],
).add_selection(selection).properties(height=250, width=250)

#Speed Chart
Speed_Bar = alt.Chart(pkmn_data).mark_bar().encode(
    y = "mean(Speed)",
    x = alt.X("Type 1", title ="Primary Type"),
    color=alt.Color("Type 1", legend=None, scale=alt.Scale(range=list(colors.values()))),
    tooltip=["Type 1", "mean(Speed)"],
).add_selection(selection).properties(height=250, width=250)

alt.vconcat(
    alt.hconcat(HP_Bar, Atk_Bar, Def_Bar),
    alt.hconcat(SpAtk_Bar, SpDef_Bar, Speed_Bar)
)

/Users/ryandadmun/venv/lib/python3.12/site-packages/altair/utils/deprecation.py:65: AltairDeprecationWarning: 'selection' is deprecated.
   Use 'selection_point()' or 'selection_interval()' instead; these functions also include more helpful docstrings.
  warnings.warn(message, AltairDeprecationWarning, stacklevel=1)
/Users/ryandadmun/venv/lib/python3.12/site-packages/altair/vegalite/v5/api.py:469: AltairDeprecationWarning: The types 'single' and 'multi' are now
        combined and should be specified using "selection_point()".
  warnings.warn(
/Users/ryandadmun/venv/lib/python3.12/site-packages/altair/utils/deprecation.py:65: AltairDeprecationWarning: 'add_selection' is deprecated. Use 'add_params' instead.
  warnings.warn(message, AltairDeprecationWarning, stacklevel=1)


alt.VConcatChart(...)

Although this is pretty good, maybe Box and Whisker charts would be more effective for displaying this information:

In [ ]:
# Box and Whisker Plots
selection_HP = alt.selection(type="multi", fields=["Type 1"], name="selection_HP")
selection_Atk = alt.selection(type="multi", fields=["Type 1"], name="selection_Atk")
selection_Def = alt.selection(type="multi", fields=["Type 1"], name="selection_Def")
selection_SpAtk = alt.selection(type="multi", fields=["Type 1"], name="selection_SpAtk")
selection_SpDef = alt.selection(type="multi", fields=["Type 1"], name="selection_SpDef")
selection_Speed = alt.selection(type="multi", fields=["Type 1"], name="selection_Speed")

#HP_Box
box_HP = alt.Chart(pkmn_data).mark_boxplot().encode(
    x='Type 1:N',
    y='HP:Q',
    color=alt.Color("Type 1", legend=None, scale=alt.Scale(range=list(colors.values()))),
).add_selection(selection_HP).properties(
    title = 'Box and Whisker Plot of Pokemon HP by Type',
    height=250,
    width=250)

#Attack_Box
box_Atk = alt.Chart(pkmn_data).mark_boxplot().encode(
    x='Type 1:N',
    y='Attack:Q',
    color=alt.Color("Type 1", legend=None, scale=alt.Scale(range=list(colors.values()))),
).add_selection(selection_Atk).properties(
    title = 'Box and Whisker Plot of Pokemon Attack by Type',
    height=250,
    width=250)

#Defense_Box
box_Def = alt.Chart(pkmn_data).mark_boxplot().encode(
    x='Type 1:N',
    y='Defense:Q',
    color=alt.Color("Type 1", legend=None, scale=alt.Scale(range=list(colors.values()))),
).add_selection(selection_Def).properties(
    title = 'Box and Whisker Plot of Pokemon Defense by Type',
    height=250,
    width=250)

#Sp_Atk_Box
box_SpAtk = alt.Chart(pkmn_data).mark_boxplot().encode(
    x='Type 1:N',
    y='Sp_Atk:Q',
    color=alt.Color("Type 1", legend=None, scale=alt.Scale(range=list(colors.values()))),
).add_selection(selection_SpAtk).properties(
    title = 'Box and Whisker Plot of Pokemon Sp. Atk by Type',
    height=250,
    width=250)

#Sp_Def_Box
box_SpDef = alt.Chart(pkmn_data).mark_boxplot().encode(
    x='Type 1:N',
    y='Sp_Def:Q',
    color=alt.Color("Type 1", legend=None, scale=alt.Scale(range=list(colors.values()))),
).add_selection(selection_SpDef).properties(
    title = 'Box and Whisker Plot of Pokemon Sp. Def by Type',
    height=250,
    width=250)

#Speed_Box
box_Speed = alt.Chart(pkmn_data).mark_boxplot().encode(
    x='Type 1:N',
    y='Speed:Q',
    color=alt.Color("Type 1", legend=None, scale=alt.Scale(range=list(colors.values()))),
).add_selection(selection_Speed).properties(
    title = 'Box and Whisker Plot of Pokemon Speed by Type',
    height=250,
    width=250)

box_HP.display()
box_Atk.display()
box_Def.display()
box_SpAtk.display()
box_SpDef.display()
box_Speed.display()

/Users/ryandadmun/venv/lib/python3.12/site-packages/altair/utils/deprecation.py:65: AltairDeprecationWarning: 'selection' is deprecated.
   Use 'selection_point()' or 'selection_interval()' instead; these functions also include more helpful docstrings.
  warnings.warn(message, AltairDeprecationWarning, stacklevel=1)
/Users/ryandadmun/venv/lib/python3.12/site-packages/altair/vegalite/v5/api.py:469: AltairDeprecationWarning: The types 'single' and 'multi' are now
        combined and should be specified using "selection_point()".
  warnings.warn(
/Users/ryandadmun/venv/lib/python3.12/site-packages/altair/utils/deprecation.py:65: AltairDeprecationWarning: 'add_selection' is deprecated. Use 'add_params' instead.
  warnings.warn(message, AltairDeprecationWarning, stacklevel=1)


alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

This is pretty helpful in directly comparing the stat differences per type, but hopefully we can make a single graph to highlight this in a single visualization.  

In [ ]:
#First, we need to build a function to group by
def alt_stats_by(classifier):
    stats_names = ['HP', 'Attack', 'Defense', 'Sp_Atk', 'Sp_Def', 'Speed']
    
    # Group by classifier and calculate mean
    stats_mean = pkmn_data.groupby(classifier)[stats_names].mean().reset_index()


    # Melt the dataframe for Altair's long-form data requirement
    melted_stats_mean = stats_mean.melt(id_vars=[classifier], value_vars=stats_names, var_name='Stat', value_name='MeanValue')


    # Create our chart
    chart = alt.Chart(melted_stats_mean).mark_line(point=True).encode(
        x=alt.X(f'{classifier}:N', title=f'{classifier}'),
        y=alt.Y('MeanValue:Q', title='Mean Values', scale=alt.Scale(domain=[40,120])),
        color='Stat:N',
        tooltip=[alt.Tooltip(f'{classifier}:N', title=f'{classifier}'), 'Stat:N', 'MeanValue:Q']
    ).properties(
        title=f'Mean Trend of Stats by {classifier}',
        width=600,
        height=400
    )

    return chart

# Visualize trend of stats by Generation and Type 1
alt_stats_by('Type 1')

alt.Chart(...)

Much better!

Let's also make a similar graph using the Generation of pokemon as our classifier to see how stats changed throughout the series. 

In [ ]:
#First, we need to build a function to group by
def alt_stats_by(classifier):
    stats_names = ['HP', 'Attack', 'Defense', 'Sp_Atk', 'Sp_Def', 'Speed']
    
    # Group by classifier and calculate mean
    stats_mean = pkmn_data.groupby(classifier)[stats_names].mean().reset_index()


    # Melt the dataframe for Altair's long-form data requirement
    melted_stats_mean = stats_mean.melt(id_vars=[classifier], value_vars=stats_names, var_name='Stat', value_name='MeanValue')


    # Create our chart
    chart = alt.Chart(melted_stats_mean).mark_line(point=True).encode(
        x=alt.X(f'{classifier}:N', title=f'{classifier}'),
        y=alt.Y('MeanValue:Q', title='Mean Values', scale=alt.Scale(domain=[60,85])),
        color='Stat:N',
        tooltip=[alt.Tooltip(f'{classifier}:N', title=f'{classifier}'), 'Stat:N', 'MeanValue:Q']
    ).properties(
        title=f'Mean Trend of Stats by {classifier}',
        width=600,
        height=400
    )

    return chart

# Visualize trend of stats by Generation and Type 1
alt_stats_by('Generation')

alt.Chart(...)

And now for my final visualization, I'd like to present 36 scatterplots which compare statistics against each other.

In [ ]:
#HP Graphs
HP_HP = alt.Chart(pkmn_data).mark_circle(size=60).encode(
    x='HP:Q',
    y='HP:Q',
    color=alt.Color("Type 1", scale=alt.Scale(range=list(colors.values()))),
    tooltip=['Name:N', 'HP:Q', 'HP:Q', 'Type 1:N']
).properties(
    title='Scatter Plot of HP vs HP Scores',
    width=200,
    height=200
)

HP_Atk = alt.Chart(pkmn_data).mark_circle(size=60).encode(
    x='HP:Q',
    y='Attack:Q',
    color=alt.Color("Type 1", scale=alt.Scale(range=list(colors.values()))),
    tooltip=['Name:N', 'HP:Q', 'Attack:Q', 'Type 1:N']
).properties(
    title='Scatter Plot of HP vs Attack Scores',
    width=200,
    height=200
)

HP_Def = alt.Chart(pkmn_data).mark_circle(size=60).encode(
    x='HP:Q',
    y='Defense:Q',
    color=alt.Color("Type 1", scale=alt.Scale(range=list(colors.values()))),
    tooltip=['Name:N', 'HP:Q', 'Defense:Q', 'Type 1:N']
).properties(
    title='Scatter Plot of HP vs Defense Scores',
    width=200,
    height=200
)

HP_SpAtk = alt.Chart(pkmn_data).mark_circle(size=60).encode(
    x='HP:Q',
    y='Sp_Atk:Q',
    color=alt.Color("Type 1", scale=alt.Scale(range=list(colors.values()))),
    tooltip=['Name:N', 'HP:Q', 'Sp_Atk:Q', 'Type 1:N']
).properties(
    title='Scatter Plot of HP vs Sp. Atk Scores',
    width=200,
    height=200 
)

HP_SpDef = alt.Chart(pkmn_data).mark_circle(size=60).encode(
    x='HP:Q',
    y='Sp_Def:Q',
    color=alt.Color("Type 1", scale=alt.Scale(range=list(colors.values()))),
    tooltip=['Name:N', 'HP:Q', 'Sp_Def:Q', 'Type 1:N']
).properties(
    title='Scatter Plot of HP vs Sp. Def Scores',
    width=200,
    height=200
)

HP_Speed = alt.Chart(pkmn_data).mark_circle(size=60).encode(
    x='HP:Q',
    y='Speed:Q',
    color=alt.Color("Type 1", scale=alt.Scale(range=list(colors.values()))),
    tooltip=['Name:N', 'HP:Q', 'Speed:Q', 'Type 1:N']
).properties(
    title='Scatter Plot of HP vs Speed Scores',
    width=200,
    height=200
)

#Atk Graphs
Atk_HP = alt.Chart(pkmn_data).mark_circle(size=60).encode(
    x='Attack:Q',
    y='HP:Q',
    color=alt.Color("Type 1", scale=alt.Scale(range=list(colors.values()))),
    tooltip=['Name:N', 'Attack:Q', 'HP:Q', 'Type 1:N']
).properties(
    title='Scatter Plot of Atk vs HP Scores',
    width=200,
    height=200
)

Atk_Atk = alt.Chart(pkmn_data).mark_circle(size=60).encode(
    x='Attack:Q',
    y='Attack:Q',
    color=alt.Color("Type 1", scale=alt.Scale(range=list(colors.values()))),
    tooltip=['Name:N', 'Attack:Q', 'Attack:Q', 'Type 1:N']
).properties(
    title='Scatter Plot of Attack vs Attack Scores',
    width=200,
    height=200
)

Atk_Def = alt.Chart(pkmn_data).mark_circle(size=60).encode(
    x='Attack:Q',
    y='Defense:Q',
    color=alt.Color("Type 1", scale=alt.Scale(range=list(colors.values()))),
    tooltip=['Name:N', 'Attack:Q', 'Defense:Q', 'Type 1:N']
).properties(
    title='Scatter Plot of Attack vs Defense Scores',
    width=200,
    height=200
)

Atk_SpAtk = alt.Chart(pkmn_data).mark_circle(size=60).encode(
    x='Attack:Q',
    y='Sp_Atk:Q',
    color=alt.Color("Type 1", scale=alt.Scale(range=list(colors.values()))),
    tooltip=['Name:N', 'Attack:Q', 'Sp_Atk:Q', 'Type 1:N']
).properties(
    title='Scatter Plot of Attack vs Sp. Atk Scores',
    width=200,
    height=200 
)

Atk_SpDef = alt.Chart(pkmn_data).mark_circle(size=60).encode(
    x='Attack:Q',
    y='Sp_Def:Q',
    color=alt.Color("Type 1", scale=alt.Scale(range=list(colors.values()))),
    tooltip=['Name:N', 'Attack:Q', 'Sp_Def:Q', 'Type 1:N']
).properties(
    title='Scatter Plot of Attack vs Sp. Def Scores',
    width=200,
    height=200
)

Atk_Speed = alt.Chart(pkmn_data).mark_circle(size=60).encode(
    x='Attack:Q',
    y='Speed:Q',
    color=alt.Color("Type 1", scale=alt.Scale(range=list(colors.values()))),
    tooltip=['Name:N', 'Attack:Q', 'Speed:Q', 'Type 1:N']
).properties(
    title='Scatter Plot of Attack vs Speed Scores',
    width=200,
    height=200
)

#Defense Graphs
Def_HP = alt.Chart(pkmn_data).mark_circle(size=60).encode(
    x='Defense:Q',
    y='HP:Q',
    color=alt.Color("Type 1", scale=alt.Scale(range=list(colors.values()))),
    tooltip=['Name:N', 'Defense:Q', 'HP:Q', 'Type 1:N']
).properties(
    title='Scatter Plot of Defense vs HP Scores',
    width=200,
    height=200
)

Def_Atk = alt.Chart(pkmn_data).mark_circle(size=60).encode(
    x='Defense:Q',
    y='Attack:Q',
    color=alt.Color("Type 1", scale=alt.Scale(range=list(colors.values()))),
    tooltip=['Name:N', 'Defense:Q', 'Attack:Q', 'Type 1:N']
).properties(
    title='Scatter Plot of Defense vs Attack Scores',
    width=200,
    height=200
)

Def_Def = alt.Chart(pkmn_data).mark_circle(size=60).encode(
    x='Defense:Q',
    y='Defense:Q',
    color=alt.Color("Type 1", scale=alt.Scale(range=list(colors.values()))),
    tooltip=['Name:N', 'Defense:Q', 'Defense:Q', 'Type 1:N']
).properties(
    title='Scatter Plot of Defense vs Defense Scores',
    width=200,
    height=200
)

Def_SpAtk = alt.Chart(pkmn_data).mark_circle(size=60).encode(
    x='Defense:Q',
    y='Sp_Atk:Q',
    color=alt.Color("Type 1", scale=alt.Scale(range=list(colors.values()))),
    tooltip=['Name:N', 'Defense:Q', 'Sp_Atk:Q', 'Type 1:N']
).properties(
    title='Scatter Plot of Defense vs Sp. Atk Scores',
    width=200,
    height=200 
)

Def_SpDef = alt.Chart(pkmn_data).mark_circle(size=60).encode(
    x='Defense:Q',
    y='Sp_Def:Q',
    color=alt.Color("Type 1", scale=alt.Scale(range=list(colors.values()))),
    tooltip=['Name:N', 'Defense:Q', 'Sp_Def:Q', 'Type 1:N']
).properties(
    title='Scatter Plot of Defense vs Sp. Def Scores',
    width=200,
    height=200
)

Def_Speed = alt.Chart(pkmn_data).mark_circle(size=60).encode(
    x='Defense:Q',
    y='Speed:Q',
    color=alt.Color("Type 1", scale=alt.Scale(range=list(colors.values()))),
    tooltip=['Name:N', 'Defense:Q', 'Speed:Q', 'Type 1:N']
).properties(
    title='Scatter Plot of Defense vs Speed Scores',
    width=200,
    height=200
)

#Sp. Atk Graphs
SpAtk_HP = alt.Chart(pkmn_data).mark_circle(size=60).encode(
    x='Sp_Atk:Q',
    y='HP:Q',
    color=alt.Color("Type 1", scale=alt.Scale(range=list(colors.values()))),
    tooltip=['Name:N', 'Sp_Atk:Q', 'HP:Q', 'Type 1:N']
).properties(
    title='Scatter Plot of Sp. Atk vs HP Scores',
    width=200,
    height=200
)

SpAtk_Atk = alt.Chart(pkmn_data).mark_circle(size=60).encode(
    x='Sp_Atk:Q',
    y='Attack:Q',
    color=alt.Color("Type 1", scale=alt.Scale(range=list(colors.values()))),
    tooltip=['Name:N', 'Sp_Atk:Q', 'Attack:Q', 'Type 1:N']
).properties(
    title='Scatter Plot of Sp. Atk vs Attack Scores',
    width=200,
    height=200
)

SpAtk_Def = alt.Chart(pkmn_data).mark_circle(size=60).encode(
    x='Sp_Atk:Q',
    y='Defense:Q',
    color=alt.Color("Type 1", scale=alt.Scale(range=list(colors.values()))),
    tooltip=['Name:N', 'Sp_Atk:Q', 'Defense:Q', 'Type 1:N']
).properties(
    title='Scatter Plot of Sp. Atk vs Defense Scores',
    width=200,
    height=200
)

SpAtk_SpAtk = alt.Chart(pkmn_data).mark_circle(size=60).encode(
    x='Sp_Atk:Q',
    y='Sp_Atk:Q',
    color=alt.Color("Type 1", scale=alt.Scale(range=list(colors.values()))),
    tooltip=['Name:N', 'Sp_Atk:Q', 'Sp_Atk:Q', 'Type 1:N']
).properties(
    title='Scatter Plot of Sp. Atk vs Sp. Atk Scores',
    width=200,
    height=200 
)

SpAtk_SpDef = alt.Chart(pkmn_data).mark_circle(size=60).encode(
    x='Sp_Atk:Q',
    y='Sp_Def:Q',
    color=alt.Color("Type 1", scale=alt.Scale(range=list(colors.values()))),
    tooltip=['Name:N', 'Sp_Atk:Q', 'Sp_Def:Q', 'Type 1:N']
).properties(
    title='Scatter Plot of Sp. Atk vs Sp. Def Scores',
    width=200,
    height=200
)

SpAtk_Speed = alt.Chart(pkmn_data).mark_circle(size=60).encode(
    x='Sp_Atk:Q',
    y='Speed:Q',
    color=alt.Color("Type 1", scale=alt.Scale(range=list(colors.values()))),
    tooltip=['Name:N', 'Sp_Atk:Q', 'Speed:Q', 'Type 1:N']
).properties(
    title='Scatter Plot of Sp. Atk vs Speed Scores',
    width=200,
    height=200
)

#Sp. Defense Graphs
SpDef_HP = alt.Chart(pkmn_data).mark_circle(size=60).encode(
    x='Sp_Def:Q',
    y='HP:Q',
    color=alt.Color("Type 1", scale=alt.Scale(range=list(colors.values()))),
    tooltip=['Name:N', 'Sp_Def:Q', 'HP:Q', 'Type 1:N']
).properties(
    title='Scatter Plot of Sp. Def vs HP Scores',
    width=200,
    height=200
)

SpDef_Atk = alt.Chart(pkmn_data).mark_circle(size=60).encode(
    x='Sp_Def:Q',
    y='Attack:Q',
    color=alt.Color("Type 1", scale=alt.Scale(range=list(colors.values()))),
    tooltip=['Name:N', 'Sp_Def:Q', 'Attack:Q', 'Type 1:N']
).properties(
    title='Scatter Plot of Sp. Def vs Attack Scores',
    width=200,
    height=200
)

SpDef_Def = alt.Chart(pkmn_data).mark_circle(size=60).encode(
    x='Sp_Def:Q',
    y='Defense:Q',
    color=alt.Color("Type 1", scale=alt.Scale(range=list(colors.values()))),
    tooltip=['Name:N', 'Sp_Def:Q', 'Defense:Q', 'Type 1:N']
).properties(
    title='Scatter Plot of Sp. Def vs Defense Scores',
    width=200,
    height=200
)

SpDef_SpAtk = alt.Chart(pkmn_data).mark_circle(size=60).encode(
    x='Sp_Def:Q',
    y='Sp_Atk:Q',
    color=alt.Color("Type 1", scale=alt.Scale(range=list(colors.values()))),
    tooltip=['Name:N', 'Sp_Def:Q', 'Sp_Atk:Q', 'Type 1:N']
).properties(
    title='Scatter Plot of Sp. Def vs Sp. Atk Scores',
    width=200,
    height=200 
)

SpDef_SpDef = alt.Chart(pkmn_data).mark_circle(size=60).encode(
    x='Sp_Def:Q',
    y='Sp_Def:Q',
    color=alt.Color("Type 1", scale=alt.Scale(range=list(colors.values()))),
    tooltip=['Name:N', 'Sp_Def:Q', 'Sp_Def:Q', 'Type 1:N']
).properties(
    title='Scatter Plot of Sp. Def vs Sp. Def Scores',
    width=200,
    height=200
)

SpDef_Speed = alt.Chart(pkmn_data).mark_circle(size=60).encode(
    x='Sp_Def:Q',
    y='Speed:Q',
    color=alt.Color("Type 1", scale=alt.Scale(range=list(colors.values()))),
    tooltip=['Name:N', 'Sp_Def:Q', 'Speed:Q', 'Type 1:N']
).properties(
    title='Scatter Plot of Sp. Def vs Speed Scores',
    width=200,
    height=200
)

#Speed Graphs
Speed_HP = alt.Chart(pkmn_data).mark_circle(size=60).encode(
    x='Speed:Q',
    y='HP:Q',
    color=alt.Color("Type 1", scale=alt.Scale(range=list(colors.values()))),
    tooltip=['Name:N', 'Speed:Q', 'HP:Q', 'Type 1:N']
).properties(
    title='Scatter Plot of Speed vs HP Scores',
    width=200,
    height=200
)

Speed_Atk = alt.Chart(pkmn_data).mark_circle(size=60).encode(
    x='Speed:Q',
    y='Attack:Q',
    color=alt.Color("Type 1", scale=alt.Scale(range=list(colors.values()))),
    tooltip=['Name:N', 'Speed:Q', 'Attack:Q', 'Type 1:N']
).properties(
    title='Scatter Plot of Speed vs Attack Scores',
    width=200,
    height=200
)

Speed_Def = alt.Chart(pkmn_data).mark_circle(size=60).encode(
    x='Speed:Q',
    y='Defense:Q',
    color=alt.Color("Type 1", scale=alt.Scale(range=list(colors.values()))),
    tooltip=['Name:N', 'Speed:Q', 'Defense:Q', 'Type 1:N']
).properties(
    title='Scatter Plot of Speed vs Defense Scores',
    width=200,
    height=200
)

Speed_SpAtk = alt.Chart(pkmn_data).mark_circle(size=60).encode(
    x='Speed:Q',
    y='Sp_Atk:Q',
    color=alt.Color("Type 1", scale=alt.Scale(range=list(colors.values()))),
    tooltip=['Name:N', 'Speed:Q', 'Sp_Atk:Q', 'Type 1:N']
).properties(
    title='Scatter Plot of Speed vs Sp. Atk Scores',
    width=200,
    height=200 
)

Speed_SpDef = alt.Chart(pkmn_data).mark_circle(size=60).encode(
    x='Speed:Q',
    y='Sp_Def:Q',
    color=alt.Color("Type 1", scale=alt.Scale(range=list(colors.values()))),
    tooltip=['Name:N', 'Speed:Q', 'Sp_Def:Q', 'Type 1:N']
).properties(
    title='Scatter Plot of Speed vs Sp. Def Scores',
    width=200,
    height=200
)

Speed_Speed = alt.Chart(pkmn_data).mark_circle(size=60).encode(
    x='Speed:Q',
    y='Speed:Q',
    color=alt.Color("Type 1", scale=alt.Scale(range=list(colors.values()))),
    tooltip=['Name:N', 'Speed:Q', 'Speed:Q', 'Type 1:N']
).properties(
    title='Scatter Plot of Speed vs Speed Scores',
    width=200,
    height=200
)


(HP_HP | HP_Atk | HP_Def | HP_SpAtk | HP_SpDef | HP_Speed) & (Atk_HP | Atk_Atk | Atk_Def | Atk_SpAtk | Atk_SpDef | Atk_Speed) & (Def_HP | Def_Atk | Def_Def | Def_SpAtk | Def_SpDef | Def_Speed) & (SpAtk_HP | SpAtk_Atk | SpAtk_Def | SpAtk_SpAtk | SpAtk_SpDef | SpAtk_Speed) & (SpDef_HP | SpDef_Atk | SpDef_Def | SpDef_SpAtk | SpDef_SpDef | SpDef_Speed) & (Speed_HP | Speed_Atk | Speed_Def | Speed_SpAtk | Speed_SpDef | Speed_Speed)

alt.VConcatChart(...)